# **SAR Target/Object Detection Using Machine Learning**

The notebook represents real time implementation of data processing, model building and testing of multiple Machine Learning Algorithms for detection of the target in MSTAR dataset.

**Necassary Packages Installation**

In [ ]:
!pip install scipy==1.2.1

     |████████████████████████████████| 24.8MB 128kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
import numpy as np
from PIL import Image
import scipy.misc as im

import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from skimage.transform import resize
from skimage import data

In [ ]:
import PIL #resizing the images

**Data Processing**

![alt text](https://drive.google.com/uc?export=view&id=11u7TILbn_1a5OvI97Zrv4nzGke8LSC88)

![alt text](https://drive.google.com/uc?export=view&id=1LtSjZGAiqjl6UPZYBJjiJupzPyyA6YGN)

In [ ]:
def get_mstar_data(stage, width=128, height=128, crop_size=128, aug=False):
    data_dir = "/content/gdrive/My Drive/PSIT Files/MSTAR_Machine_learning/MSTAR-10/train/" if stage == "train" else "/content/gdrive/My Drive/PSIT Files/MSTAR_Machine_learning/MSTAR-10/test/" if stage == "test" else None
    print("------ " + stage + " ------")
    sub_dir = ["2S1", "BMP2", "BRDM_2", "BTR60", "BTR70", "D7", "T62", "T72", "ZIL131", "ZSU_23_4"]
    X = []
    y = []

    for i in range(len(sub_dir)):
        tmp_dir = data_dir + sub_dir[i] + "/"
        img_idx = [x for x in os.listdir(tmp_dir) if x.endswith(".jpeg")]
        print(sub_dir[i], len(img_idx))
        y += [i] * len(img_idx)
        for j in range(len(img_idx)):
            img = im.imresize(im.imread((tmp_dir + img_idx[j])), [height, width])
            img = img[(height - crop_size) // 2 : height - (height - crop_size) // 2, \
                  (width - crop_size) // 2: width - (width - crop_size) // 2]
            # img = img[16:112, 16:112]   # crop
            X.append(img)

    return np.asarray(X), np.asarray(y)

In [ ]:
def data_shuffle(X, y, seed=0):
    data = np.hstack([X, y[:, np.newaxis]])
    np.random.shuffle(data)
    return data[:, :-1], data[:, -1]

In [ ]:
def one_hot(y_train, y_test):
    one_hot_trans = OneHotEncoder().fit(y_train[:, np.newaxis])
    return one_hot_trans.transform(y_train[:, np.newaxis]).toarray(), one_hot_trans.transform(y_test[:, np.newaxis]).toarray()

In [ ]:
def mean_wise(X):
    return (X.T - np.mean(X, axis=1)).T

**Principal Component Analysis for Optimized Processing**

In [ ]:
def pca(X_train, X_test, n):
    pca_trans = PCA(n_components=n).fit(X_train)
    return pca_trans.transform(X_train), pca_trans.transform(X_test)

**Analyzing Various Machine Learning Algorithms**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

**Decision Tree Classifier**

In [ ]:
def dt(criterion="entropy", max_features="sqrt"):
    return DecisionTreeClassifier(criterion=criterion, max_features=max_features, max_depth=None, random_state=0)

**Random Forest Classification**

In [ ]:
def rf(n_tree=100, max_features="sqrt"):
    return RandomForestClassifier(n_estimators=n_tree, max_features=max_features, min_samples_split=2, \
                                  max_depth=None, bootstrap=True, oob_score=False, random_state=0, n_jobs=4)

**Gradient Boosting Classification**

In [ ]:
def gbdt(n_tree=100, max_features="sqrt"):
    return GradientBoostingClassifier(n_estimators=n_tree, learning_rate=0.005, \
                                      max_features=max_features, max_depth=None, random_state=0)

**Logistic Regression**

In [ ]:
def logit(C=1.0):
    return LogisticRegression(C=1.0, solver="lbfgs", max_iter=1000, random_state=0)

**Multi-layer Perceptron Classifier**

In [ ]:
def mlp(hidden=(100), act="logistic", batch=32):
    return MLPClassifier(hidden_layer_sizes=hidden, activation=act, solver="sgd", batch_size=batch, \
                         learning_rate="constant", learning_rate_init=0.1, early_stopping=False, max_iter=1000, random_state=0)

**Support Vector Machine Classifier**

![alt text](https://drive.google.com/uc?export=view&id=1ABtElBwMDn1dmSOenCGok6i5rhsaq4Ql)

In [ ]:
def svm(C=1.0, kernel="rbf"):
    return SVC(C=C, kernel=kernel, max_iter=-1, random_state=0)

**K Nearest Neighbours Classification**

In [ ]:
def knn(n_neighbors=10, weights="distance"):
    return KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, algorithm="auto")

**Naive Bayes Classification**

In [ ]:
def bayes():
    return GaussianNB()

**Trainer Fuction**

In [ ]:
def train(X, y, classifier):
    return classifier.fit(X, y)

**Testing Function**

In [ ]:
def test(X, classifier):
    return classifier.predict(X)

**Accuracy Function**

In [ ]:
def acc(X, y, classifier):
    return classifier.score(X, y)

**Implementation of Algorithms real time in order to find the most optimized solution**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

*Let's load the processed M-STAR dataset*

In [ ]:
print("loading ... ")
X_train, y_train = get_mstar_data("train", 128, 128, 96)
X_test, y_test = get_mstar_data("test", 128, 128, 96)
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[2]])
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[2]])
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

loading ... 
------ train ------
2S1 299


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  


BMP2 233
BRDM_2 298
BTR60 256
BTR70 233
D7 299
T62 298
T72 232
ZIL131 299
ZSU_23_4 299
------ test ------
2S1 274
BMP2 195
BRDM_2 274
BTR60 195
BTR70 196
D7 274
T62 273
T72 196
ZIL131 274
ZSU_23_4 274
(2746, 9216) (2746,) (2425, 9216) (2425,)


*Shuffling the loaded dataset to eliminate any bias*

In [ ]:
print("shuffling ... ")
X_train, y_train = data_shuffle(X_train, y_train)
X_test, y_test = data_shuffle(X_test, y_test)

shuffling ... 


*Preprocessing the dataset*

In [ ]:
print("preprocessing ...")
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = mean_wise(X_train)
X_test = mean_wise(X_test)
X_train, X_test = pca(X_train, X_test, 80)
# y_train, y_test = data.one_hot(y_train, y_test)

preprocessing ...


*Finally Training the model*

In [ ]:
print("training ...")
# classifier = train(X_train, y_train, dt("entropy", 0.8)) # 70.68%
# classifier = train(X_train, y_train, rf(1000, "sqrt")) # 96.49%
# classifier = train(X_train, y_train, gbdt(1000, "sqrt")) # 95.17%
# classifier = train(X_train, y_train, logit(1.0))    # 90.14%
# classifier = train(X_train, y_train, mlp(1000, "logistic"))   # 93.36%
# classifier = train(X_train, y_train, svm(1.0, "rbf"))     # 96.82%
# classifier = train(X_train, y_train, knn(10, "uniform"))   # 95.34%
classifier = train(X_train, y_train, svm())   # 97.81%

training ...


*As seen while analyzing the above algorithms, Support Vector machine was able to provide higest accuracy for determining the vehicle images from the test data, that is 97.8% accuracy.*

In [ ]:
print("testing ...")
print(acc(X_train, y_train, classifier))
print(acc(X_test, y_test, classifier))

testing ...
0.9996358339402768
0.9781443298969072


**The END**

**Thank You**